In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.runnables import RunnableSequence, RunnablePassthrough
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv
import os

load_dotenv()
api_key_01 = os.getenv("HUGGINGFACE_API_KEY")
api_key_02 = os.getenv("GOOGLE_API_KEY")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key_01,
    temperature=0.1 # Lower temperature for better JSON generation
)

model_01 = ChatHuggingFace(llm=llm) 

model_02 = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.1, api_key=api_key_02)  

str_parser = StrOutputParser()

prompt_01 = PromptTemplate(
    template="Write a joke relevant to the following topic: {topic}",
    input_variables=["topic"]
)

prompt_02 = PromptTemplate(
    template="Expin the following joke: {joke}",
    input_variables=["joke"]
)

joke_gen_chain = RunnableSequence(
    prompt_01 | model_01 | str_parser
)

parallel_chain = RunnableParallel(
    {
        'joke': RunnablePassthrough(),
        'explanation': RunnableSequence(prompt_02 | model_02 | str_parser)
    }
)

final_chain = RunnableSequence(
    joke_gen_chain | parallel_chain
)

result = final_chain.invoke({"topic": "Artificial Intelligence"})

print(result)

{'joke': 'Why did the AI program go to therapy?\n\nBecause it was struggling to process its emotions.', 'explanation': 'This joke plays on the double meaning of the word "process" and the concept of therapy.\n\nHere\'s the breakdown:\n\n1.  **Human Context (Therapy):** When humans go to therapy, they often do so because they are "struggling to process their emotions." This means they are having difficulty understanding, coping with, integrating, or dealing with complex feelings like sadness, anger, anxiety, etc. Therapy helps them work through these internal states.\n\n2.  **AI/Computer Context:**\n    *   **"Process" (literal):** For an AI or any computer program, "processing" is its fundamental function. It means to perform operations on data, to analyze, compute, or transform information.\n    *   **"Emotions" (data):** While AI doesn\'t *feel* emotions in the human sense, it can be programmed to *detect* and *analyze* emotional data (e.g., sentiment analysis in text, facial express

In [4]:
final_chain.get_graph().print_ascii()

                        +-------------+                     
                        | PromptInput |                     
                        +-------------+                     
                                *                           
                                *                           
                                *                           
                       +----------------+                   
                       | PromptTemplate |                   
                       +----------------+                   
                                *                           
                                *                           
                                *                           
                      +-----------------+                   
                      | ChatHuggingFace |                   
                      +-----------------+                   
                                *                           
                        